In [2]:
import serial
#import matplotlib as plt
import sys
import visa
from LQCE.zvl13 import *
from LQCE.ncs513 import *
from LQCE.N5173B import *
from LQCE.awg33500 import *
import matplotlib.pyplot as plt
import numpy as np
import math
import matplotlib.cm as cm
from matplotlib.backends.backend_pdf import PdfPages
import time
import scipy.signal
%matplotlib 
import pickle

Using matplotlib backend: Qt5Agg


# __Novosibirsk current source init__

In [3]:
import socket
import crcmod
from ctypes import (Union, Array, c_uint8, c_float, cdll, CDLL)
from enum import Enum
from ctypes import *

class uint8_array(Array):
    _type_ = c_uint8
    _length_ = 4
class f_type(Union):
    _fields_ = ("float", c_float), ("char", uint8_array)

def crc8_custom(data, len = 9):
    crc = c_uint8(0xff)
    i = c_uint32(0)
    for j in range(len):
        dt = c_uint8(data[j])
        crc.value ^= dt.value;
        for k in range(8):
            if (crc.value&0x80) != 0:
                crc.value = (crc.value<<1)^0x31
            else:
                crc.value <<=1 # crc.value<<1
    return crc.value
def float_to4bytes(float_data):
    temp_data = f_type()
    temp_data.float = float_data
    return temp_data.char[:]
def byteArray_toFloat(data_array, offset = 0):
    temp_data = f_type()
    temp_data.char[:] = (data_array[0+offset],data_array[1+offset],data_array[2+offset],data_array[3+offset])
    return temp_data.float
        
def ch_output_enable(channel = 1):
    tData = [0xc1,0x70,0x71,0x71,0x71,0x71,0x71,0x7f,0x7f,0x7f]
    tData[1] = 0x70 | (channel)
    tData[9] = crc8_custom(tData, 9)
    sock.send(bytearray(tData))
    return(sock.recv(10))
def ch_output_disable(channel = 1):
    tData = [0xc1,0x70,0x70,0x70,0x70,0x70,0x70,0x7f,0x7f,0x7f]
    tData[1] = 0x70 | (channel)
    tData[9] = crc8_custom(tData, 9)
    sock.send(bytearray(tData))
    return(sock.recv(10)) 

def ch_range(channel = 1,channel_range = '1uA'):
    #1uA 10uA 100uA 1mA 10mA 50mA
    tData = [0xc1,0x60,0x60,0x60,0x60,0x60,0x60,0x7f,0x7f,0x7f]
    tData[1] = 0x60 | channel
    if channel_range == '1uA' or channel_range == 1e-6:
        tData[channel+1] = 0x60
    elif channel_range == '10uA' or channel_range == 1e-5:
        tData[channel+1] = 0x61
    elif channel_range == '100uA' or channel_range == 1e-4:
        tData[channel+1] = 0x62
    elif channel_range == '1mA' or channel_range == 1e-3:
        tData[channel+1] = 0x63
    elif channel_range == '10mA' or channel_range == 1e-2:
        tData[channel+1] = 0x64
    elif channel_range == '50mA' or channel_range == 5e-2:
        tData[channel+1] = 0x65
    else:
        tData[channel+1] = 0x60#1uA range
    tData[9] = crc8_custom(tData, 9)
    sock.send(bytearray(tData))
    return(sock.recv(10))
def ch_set_current(channel = 1, current = 0.0):
    tFloat4bytes = []*4
    tFloat4bytes = float_to4bytes(current)
    tData = [0xc1,0xD0,0xD0,0xD0,0xD0,0xD0,0xD0,0xD0,0xD0,0x7f]
    tData[1] = 0xD0 | channel
    tData[2] = tFloat4bytes[0]
    tData[3] = tFloat4bytes[1]
    tData[4] = tFloat4bytes[2]
    tData[5] = tFloat4bytes[3]
    tData[9] = crc8_custom(tData, 9)
    sock.send(bytearray(tData))
    return (sock.recv(10))
sock = socket.socket()
sock.connect(('192.168.0.77',7))


In [22]:
ch_set_current(1,0) 
ch_output_enable(1)
ch_output_disable(1)
ch_range(1, '50mA')

b's`OK\x00\x00\x00\x00\x00.'

In [16]:
gen = N5173B_gen("TCPIP0::192.168.0.78::INSTR")
nwa = zvl13("TCPIP0::192.168.0.73::INSTR")
#gen.set_modulation_state(0)
#gen.set_frequency(6.52,"GHz")
#gen.set_power(0)
#float(gen.get_power())
#gen.set_output_state("OFF")
#gen.get_output_state()
#nwa.set_output_state("OFF")
 

In [ ]:
#data_averaged = np.average(z_upd,0)
peak_data = scipy.signal.find_peaks(z_upd, height=4, distance=100)
print(peak_data)
print(x_axis[peak_data[0][0]])#,x_axis[peak_data[0][1]],x_axis[peak_data[0][2]], x_axis[peak_data[0][3]],x_axis[peak_data[0][4]],x_axis[peak_data[0][5]])


# __Resonator frequecies__

In [59]:
#nwa = zvl13("TCPIP0::192.168.0.73::INSTR")

res1 = [6.48367e9, 30e6]#qubit 1
res2 = [6.579342e9, 50e6]#qubit 2
res3 = [6.669865e9, 20e6]
res4 = [6.758309e9,20e6]
res5 = [6.856067e9, 20e6]
res6 = [6.941001e9, 100e6]#qubit 6
res7 = [7.461176e9,20e6]#resonator
#res8 = [6.685670e9, 6e6]

# __AWG__
# _if nwa connected via mixer_

In [221]:
#awg = awg33500("TCPIP0::192.168.0.80::INSTR")
#awg.awg_init_custom_2ch([0,0], 14e-6, 15e-6)
awg.awg_init_custom_2ch(cal_result[0],14e-6, 15e-6)
awg_set_delay(-100-20)
#awg.set_voltage_offset(2, 0.05)
#awg.set_voltage_offset(1, 0.05)
#awg.set_voltage(1,100e-3)

gen.set_modulation_state(0)
gen.set_frequency(5.0e9)
gen.set_output_state("OFF")

# __ONE TONE__

In [ ]:
nwa.set_output_state("ON")
nwa.set_standard_config({"power":-70,"nop":201, "cf_span":res6, "bandwidth":100, "meas_counts":2})

ch_output_enable(3)
ch_range(3, '1mA')#available ranges: 1uA 10uA 100uA 1mA 10mA 50mA
ch_set_current(3,0) 

nwa.set_timeout(200000)

plt.ion()
fig = plt.figure()
z_upd = []
current_label = []

offset_current = 100e-6
step_current = 1e-6
current_points = 75

for i in range(current_points):
    ch_set_current(3, offset_current+i*step_current) 
    current_label.append(offset_current+i*step_current)
    nwa.trig_meas()
    x_axis = nwa.get_X_axis_values() 
    data = nwa.get_data(int(nwa.get_meas_counts()))
    x_upd = x_axis
    y_upd = []
    nx = 101#x_upd.shape[0]
    no_labels = 7
    step_x = int(nx/(no_labels -1))
    step_current_axis = int(current_points/(no_labels -1))
    x_positions = np.arange(0,nx,step_x)
    
    x_labels = x_upd[::step_x]
    z_upd.append(np.average(data[0],0))
    
    y_upd.append(i)
    plt.imshow(np.transpose(z_upd),interpolation='nearest', aspect='auto', origin = 'lower',cmap=cm.bone, extent=[offset_current*1e3,offset_current*1e3+i*step_current*1e3,x_axis[0]/1e9,x_axis[len(x_axis)-1]/1e9])
    plt.xlabel("Current, mA")
    plt.ylabel("Frequency, GHz")
    plt.title("OneTone Spectroscopy {}/{}".format(i+1,current_points))
    fig.set_size_inches(8, 6)
    fig.canvas.draw()
    fig.canvas.flush_events()
clb = plt.colorbar()
clb.set_label("S21, dBm", rotation = 0, labelpad = -40, y = 1.05)
ch_set_current(3, 0)

# __Save DATA__

In [15]:
np.save("E:\\MEASUREMENTS\\10mK_BlueFors_BFLD400\\RUN49\\{}\\ONETONE\\{}.npy".format('RES1',1), np.transpose(z_upd))


In [60]:
#save actual data to numpy file
np.save("E:\\MEASUREMENTS\\10mK_BlueFors_BFLD400\\RUN34\\res0_6.577GHz\\OneToneS_6.459GHz_101p_5Av_4uA_-40dBm_offset_-20uA", np.transpose(z_upd))
#E:\MEASUREMENTS\10mK_BlueFors_BFLD400\RUN34\res0_6.577GHz
#b = np.load("E:\\MEASUREMENTS\\10mK_BlueFors_BFLD400\\RUN33\\run33\\sweetspot_-40dBm.npy")
#plt.imshow(b, interpolation='nearest', aspect='auto', cmap=cm.RdBu)

In [53]:
fig = plt.figure()
b = np.load("E:\\MEASUREMENTS\\10mK_BlueFors_BFLD400\\RUN34\\res0_6.577GHz\\OneToneS_51p_5Av_4uA_-40dBm.npy")
plt.imshow(b, interpolation='nearest', aspect='auto', cmap=cm.RdBu)


# _TWO TONE_

In [170]:
nwa.set_output_state("ON")
nwa.set_standard_config({"power":-70,"nop":3,"averages": 1, "cf_span":[res6[0],10000], "bandwidth":100, "meas_counts":3})
nwa.set_timeout(500000)
#gen = N5173B_gen("TCPIP0::192.168.0.78::INSTR")
gen.set_modulation_state(0)
gen.set_frequency(5.0e9)
gen.set_output_state("ON")
gen.get_output_state()

gen.set_power(-20)#-20 at room

offset_current = 110e-6#105e-6#300e-6
current_list_len = 101 #number of points
step_current = 0.1e-6

gen_freq_list = 101
gen_freq_offset = 7e9
gen_freq_step = 1e6
#gen.write(":PULM:INTernal:TRAin:ONTime {:.3e}".format(900e-9))
gen.set_modulation_state(0)
plt.ion()

fig = plt.figure()
pow_slice = []

for j in range(gen_freq_list):
    gen.set_frequency(gen_freq_offset + j*gen_freq_step, "Hz")
    time.sleep(1)
    
    onetone_data = []
    nwa_slice = []
    for i in range(current_list_len):
        #ncs.set_current(3,i*20e-6)#10uA step
        ch_set_current(3,offset_current+i*step_current) 
        time.sleep(0.02)
        nwa.trig_meas()
        x_axis = nwa.get_X_axis_values()   
        data = nwa.get_data(int(nwa.get_meas_counts()))
        x_upd = x_axis#[]
        y_upd = []
        nx = 101#x_upd.shape[0]
        no_labels = 5
        step_x = int(nx/(no_labels -1))
        x_positions = np.arange(0,nx,step_x)
        x_labels = x_upd[::step_x]
        
        #z_upd.append(data[0][0])
        #z_upd.append(np.average(data[0],0))
        
        y_upd.append(i)
        #nwa_slice = data[0][0]
        #onetone_data.append(nwa_slice[2])
        nwa_slice = np.average(data[0],0)
        onetone_data.append(nwa_slice[1])

    pow_slice.append(onetone_data)
    plt.imshow(pow_slice,interpolation='nearest', aspect='auto', cmap=cm.bone, origin = 'lower', extent=[offset_current*1e3,offset_current*1e3+i*step_current*1e3,(gen_freq_offset)/1e9,(gen_freq_offset + j*gen_freq_step)/1e9])
    plt.title("Current freq - {}".format((j+1)*gen_freq_step+gen_freq_offset))
    #plt.yticks(x_positions, x_labels)
    fig.canvas.draw()
    fig.canvas.flush_events()        
plt.colorbar()
ch_set_current(3,0) 

b's\xd0OK\x00\x00\x00\x00\x00\x10'

# _SAVE_

In [130]:
np.save("E:\\MEASUREMENTS\\10mK_BlueFors_BFLD400\\RUN49\\TwoToneS_res6",pow_slice)

# _LOAD_

In [70]:
fig = plt.figure()
b = np.load("E:\\MEASUREMENTS\\10mK_BlueFors_BFLD400\\RUN34\\res0_6.577GHz\\TwoToneS_6.459GHz_7-9GHz_-40dBm_50uA_190uA_2uA.npy")
plt.imshow(b, interpolation='nearest', aspect='auto', cmap=cm.RdBu)

In [171]:

awg = awg33500("TCPIP0::192.168.0.80::INSTR")
#awg.awg_init_custom_2ch([0,0], 14e-6, 15e-6)

#awg.set_voltage_offset(2, 0.05)
#awg.set_voltage_offset(1, 0.05)
#awg.set_voltage(1,100e-3)
#awg.set_voltage(2,100e-3)
#awg.synchronize_channels()

In [98]:
awg.awg_init_custom_2ch([0,0], 3e-6, 500e-6)
#awg.synchronize_channels()
#awg.awg_set_delay(20)#in ns| for ex. (100) = 100ns (with offset 100ns = 200ns)

rabi_step = 4 # in ns
rabi_counts = 11


rabi_data_average = []
rabi_phase_average = []
r_data = []
r_phase = []
rabi_x_axis = []
temp_dB = []
temp_Phase = []


#awg.awg_set_delay(20)
#awg.synchronize_channels()


for i in range(rabi_counts):
    #awg.set_sync_output("OFF")
    #gen.set_output_state("OFF")
    #gen.set_pulse_train_ontime(500e-9 + i*1e-9*rabi_step)
    #gen.write(":PULM:INTernal:TRAin:ONTime {:.3e}".format(20e-9 + i*1e-9*rabi_step))
    #gen_init_ramsey_mode(5.805619, 20e-9 + i*1e-9*rabi_step, delay = 101e-6)
    #time.sleep(0.5)
        
    awg.awg_init_custom_2ch([0,0],10e-6, 20e-9+i*rabi_step*1e-9)
    awg.awg_set_delay(20 + i*rabi_step)
    awg.synchronize_channels()
    time.sleep(2)


In [126]:
awg.synchronize_channels()
awg.awg_init_custom_2ch([0,0],20e-9, 10e-6)
awg_set_delay(-100-20)
awg.synchronize_channels()
rabi_counts = 11
for i in range(rabi_counts):  
    awg_set_delay(-100-20-i*4) 
    time.sleep(1)

In [120]:
def awg_set_delay(delay):
    set_phase_units("SEC")
    awg.write("SOURce1:PHASe {}".format(delay*1e-9))
    awg.write("SOURce2:PHASe {}".format(delay*1e-9))
def set_phase_units(units = "DEG"):
    """DEGree | RADian | SECond """
    awg.write("UNIT:ANGLe {}".format(units))  

In [172]:
cal_result = calibrate(-40, res5[0])

C:\Users\Qubit105s\Anaconda3\lib\site-packages\scipy\optimize\_minimize.py:518: RuntimeWarning: Method Nelder-Mead cannot handle constraints nor bounds.
  RuntimeWarning)


 final_simplex: (array([[ 0.00759617, -0.00663007],
       [ 0.00759617, -0.00663007],
       [ 0.00759617, -0.00663007]]), array([-82.38403536, -69.82047872, -65.20746188]))
           fun: -82.38403535568146
       message: 'Maximum number of iterations has been exceeded.'
          nfev: 249
           nit: 100
        status: 2
       success: False
             x: array([ 0.00759617, -0.00663007])


In [176]:
def get_data_reim(data_frames_count):
    curves = [0]*data_frames_count
    out_data = []
    out_phase = []
    out_re = []
    out_im = []
    for i in range(data_frames_count):
        curves[i] = nwa.query("CALCulate1:DATA:NSWeep? SDATa, {}".format(i+1))
        np_d = np.fromstring(curves[i], sep = ',')
        temp_data = []*int(len(np_d)/2)
        temp_phase = []*int(len(np_d)/2)
        temp_re = []*int(len(np_d)/2)
        temp_im = []*int(len(np_d)/2)
        for j in range(int(len(np_d)/2)):
            temp_data.append(20*math.log(math.sqrt(np_d[2*j]**2+np_d[2*j+1]**2),10))
            temp_phase.append(math.atan2(np_d[2*j+1],np_d[2*j]))
            temp_re.append(np_d[2*j])
            temp_im.append(np_d[2*j+1])
        out_re.append(temp_re)
        out_im.append(temp_im)
    return (out_re, out_im)    

In [188]:
rabi_data_result
plt.figure()
plt.plot(rabi_data_result)

In [223]:
ch_set_current(3,0) 
[5.710691e9,20e6]

[5710691000.0, 20000000.0]

In [235]:
res4_pulse_excited = 7.625e9
re4_current = 73e-6
gen.write(":PULM:INTernal:TRAin:ONTime {:.3e}".format(20e-9 + 8*1e-9))

# __Calibration procedure__

In [ ]:
import time
import scipy.signal
import scipy.optimize
#mixer calibration part    
def find_minimum(x):
    awg.set_voltage_offset(2, x[1])
    awg.set_voltage_offset(1, x[0])
    time.sleep(0.1)
    nwa.trig_meas()
    temp_data = nwa.get_data(3)
    data_averaged = np.average(temp_data[0],0)
    #temp_slice = data_averaged[1]
    return (data_averaged[1])

def calibrate(power, freq):
    nwa.set_standard_config({"power": power,"nop":2, "cf_span":[freq, 2], "bandwidth":100, "meas_counts":10})
    nwa.set_output_state("ON")
    awg.awg_base_init()
    #gen.set_output_state("OFF")  
    awg.set_output(1, "ON")
    awg.set_output(2, "ON") 
    
    calibrate_res = scipy.optimize.minimize(find_minimum,(0.1,-0.1), method='Nelder-Mead', options = {"maxiter": 100, 'xtol': 1e-2, 'disp': True}, bounds = ((-0.5,0.5), (-0.5,0.5)))
    print(calibrate_res)
    return (calibrate_res.x, calibrate_res.fun) 


In [358]:
#rabi_data_result
#plt.figure()
#plt.plot(rabi_phase_result)
awg.set_voltage(1,100e-3)
awg.set_voltage(2,100e-3)

# __Rabi Power change__

In [371]:
awg.awg_init_custom_2ch(cal_result[0], 3e-6, 500e-6)
gen.gen_init_pulse_mode(res4_pulse_excited, 20e-9)#at this moment gen wait a trigger to start pulse
#awg.synchronize_channels()
#awg.awg_set_delay(20)#in ns| for ex. (100) = 100ns (with offset 100ns = 200ns)

average_factor = 1
average_times = 200
nwa.set_standard_config({"power":-70,"nop":2, "cf_span":[5.710691e9,10], "bandwidth":100, "meas_counts":average_times})

gen.set_power(-20)

rabi_step = 4 # in ns
rabi_counts = 101
rabi_power_step = 0.01

rabi_data_average = []
rabi_phase_average = []
r_data = []
r_phase = []
rabi_x_axis = []
temp_dB = []
temp_Phase = []

#awg.awg_set_delay(20)
#awg.synchronize_channels()

#plt.figure()
#plt.ion()
#ax1 = fig.add_subplot(2,1,1)
#ax2 = fig.add_subplot(2,1,2)

fig,axis = plt.subplots(2)
#plt.setp(axis[0].get_xticklabels(), visible=False)
fig.set_size_inches(10, 6)
axis[0].grid()
axis[1].grid()

#rabi_x_axis = np.arange(20, 20+rabi_power_step*rabi_counts, rabi_power_step)
rabi_x_axis = []

nwa.set_timeout(500000)
#for j in range(average_factor):
r_data = np.zeros(rabi_counts)
r_phase = np.zeros(rabi_counts)
rabi_data_result = np.zeros(rabi_counts)
rabi_phase_result = np.zeros(rabi_counts)
for i in range(rabi_counts):
    #awg.set_sync_output("OFF")
    #gen.set_output_state("OFF")
    #gen.set_pulse_train_ontime(500e-9 + i*1e-9*rabi_step)
    #gen.write(":PULM:INTernal:TRAin:ONTime {:.3e}".format(20e-9 + i*1e-9*rabi_step))
    #gen_init_ramsey_mode(5.805619, 20e-9 + i*1e-9*rabi_step, delay = 101e-6)
    gen.set_power(-20+i*rabi_power_step)
    #time.sleep(0.5)
        
    #awg_init_custom_2ch(cal_result[0],10e-6, 100e-6+20e-9+i*rabi_step*1e-9)
    #awg.awg_set_delay(20 + i*rabi_step)
    time.sleep(0.5)
        
    nwa.trig_meas()
    #temp_data = nwa.get_data(average_factor)
    temp_data = get_data_reim(average_times)#[real, image]

    #temp_dB = np.average(temp_data[0][0])# np.average(temp_data[0],0)
    #temp_Phase =np.average(temp_data[1][0])# np.average(temp_data[1],0)

    #rabi_data_result[i] = temp_dB[1]
    #rabi_phase_result[i] = temp_Phase[1]
    #rabi_data_average.append(temp_dB)
    #rabi_phase_average.append(temp_Phase)

    #r_data = np.average(rabi_data_average,0)
    #r_phase = np.average(rabi_phase_average,0)
    ##rabi_data_result[i] = r_data
    #rabi_phase_result[i] = r_phase
    #ax2.set_xlabel("delay, nS")
    #ax1.plot(rabi_data_result, 'b--')
    #ax1.set_ylabel("S21")
    #ax2.plot(rabi_phase_result, 'g--')
    #ax2.set_ylabel("Phase")
    #ax1.set_title("Rabi {}/{}".format(j+1, average_factor))

    temp_dB.append(np.average(temp_data[0][0]))# np.average(temp_data[0],0)
    temp_Phase.append(np.average(temp_data[1][0]))# np.average(temp_data[1],0)
   
    axis[0].set_title("Rabi {}/{}".format(i+1, rabi_counts))
    axis[0].set_xlabel("Power, dBm")

    rabi_x_axis.append(i*rabi_power_step - 20)
    
    axis[1].set_xlabel("Power, dBm")
    axis[1].plot(rabi_x_axis, temp_Phase, 'g--')
    axis[1].set_ylabel("Imag Part, a.u.")    
    axis[0].plot(rabi_x_axis, temp_dB, 'b--')
    axis[0].set_ylabel("Real Part, a.u.")    
    
    fig.canvas.draw()
    fig.canvas.flush_events()


In [286]:
fig = plt.figure()
plt.plot(rabi_phase_average)

In [281]:
fig = plt.figure()
plt.plot(rabi_data_average)
plt.plot(rabi_phase_average)

In [263]:
plt.figure()
ax1 = fig.add_subplot(2,1,1)
ax2 = fig.add_subplot(2,1,2)
ax1.plot()

[-37.57455273686062, -44.93010134236209] [0.041769006610889926, 1.0172103613189392]
[-35.987080489544205, -41.885044264400655] [0.0452909754656667, 0.7363707022173953]


In [179]:
res4_pulse_excited = 7.550e9
res5_pulse_excited = 8.49171e9
ch_set_current(3,0.153136e-3) 

b's\xd0OK\x00\x00\x00\x00\x00\x10'

# __Rabi default__

In [219]:
awg.synchronize_channels()

#awg.awg_init_custom_2ch(cal_result[0], 1e-6, 20e-6)
gen.gen_init_pulse_mode(res5_pulse_excited, 100e-9)#at this moment gen wait a trigger to start pulse
awg.synchronize_channels()
awg.awg_init_custom_2ch(cal_result[0],10e-6, 15e-6)
awg_set_delay(-100-20)
awg.synchronize_channels()
#awg.awg_set_delay(20)#in ns| for ex. (100) = 100ns (with offset 100ns = 200ns)
#awg.set_voltage(1,100e-3)
#awg.set_voltage(2,100e-3)
average_factor = 500
nwa.set_standard_config({"power":-30,"nop":2, "cf_span":[res5[0],2], "bandwidth":100, "meas_counts":average_factor})


gen.set_power(-10)

rabi_step = 20 # in ns
rabi_counts = 201

rabi_data_average = []
rabi_phase_average = []
r_data = []
r_phase = []
rabi_x_axis = []
temp_dB = []
temp_Phase = []

plt.ion()


rabi_x_axis = []

fig,axis = plt.subplots(2)
#plt.setp(axis[0].get_xticklabels(), visible=False)
fig.set_size_inches(10, 6)
axis[0].grid()
axis[1].grid()
nwa.set_timeout(500000)
#for j in range(average_factor):
r_data = np.zeros(rabi_counts)
r_phase = np.zeros(rabi_counts)
rabi_data_result = np.zeros(rabi_counts)
rabi_phase_result = np.zeros(rabi_counts)
for i in range(rabi_counts):
    #awg.set_sync_output("OFF")
    #gen.set_output_state("OFF")
    #gen.set_pulse_train_ontime(500e-9 + i*1e-9*rabi_step)
    gen.write(":PULM:INTernal:TRAin:ONTime {:.3e}".format(20e-9 + i*1e-9*rabi_step))
    #gen_init_ramsey_mode(5.805619, 20e-9 + i*1e-9*rabi_step, delay = 101e-6)
    #time.sleep(0.5)
        
    #awg_init_custom_2ch(cal_result[0],10e-6, 100e-6+20e-9+i*rabi_step*1e-9)
    #awg.awg_set_delay(20+ i*rabi_step)
    awg_set_delay(-100-20-i*rabi_step) 
    #awg.synchronize_channels()
    #time.sleep(0.5)
        
    nwa.trig_meas()
    #temp_data = nwa.get_data(average_factor)
    temp_data = get_data_reim(average_factor)

    temp_dB.append(np.average(temp_data[0][0]))# np.average(temp_data[0],0)
    temp_Phase.append(np.average(temp_data[1][0]))# np.average(temp_data[1],0)
   
    axis[0].set_title("Rabi {}/{}".format(i+1, rabi_counts))
    axis[0].set_xlabel("Pulse width, ns")

    rabi_x_axis.append(i*rabi_step+20)
    
    axis[1].set_xlabel("Pulse width, ns")
    axis[1].plot(rabi_x_axis, temp_Phase, 'g--')
    axis[1].set_ylabel("Imag Part, a.u.")    
    axis[0].plot(rabi_x_axis, temp_dB, 'b--')
    axis[0].set_ylabel("Real Part, a.u.")

    
    #plt.plot(rabi_data_average)
    fig.canvas.draw()
    fig.canvas.flush_events()

phase = [0]*rabi_counts
ampl = [0]*rabi_counts
for i in range(rabi_counts):
    phase[i] = math.atan2(temp_dB[i],temp_Phase[i])
    ampl[i] = 20*math.log(math.sqrt(temp_dB[i]**2+temp_Phase[i]**2),10)
    
#fig = plt.figure()
#plt.plot(phase)
#fig = plt.figure()
#plt.plot(ampl)
    
#rabi_data_result
#plt.figure()
#plt.plot(rabi_phase_average)

In [218]:
fig = plt.figure()
plt.plot(phase)
fig = plt.figure()
plt.plot(ampl)

In [307]:
plt.ion()

fig,axis = plt.subplots(2)


#plt.setp(ax1.get_xticklabels(), visible=False)
fig.set_size_inches(10, 6)

axis[1].set_xlabel("Pulse width, ns")
axis[0].plot(rabi_x_axis, temp_dB, 'b--')
axis[0].set_ylabel("Real Part")
axis[1].plot(rabi_x_axis, temp_Phase, 'g--')
axis[1].set_ylabel("Imag Part")

Text(0, 0.5, 'Imag Part')

In [135]:
fig = plt.figure()
b = np.load("E:\\MEASUREMENTS\\10mK_BlueFors_BFLD400\\RUN33\\run33\\CAmp\\spec_0dBm.npy")
plt.imshow(b,interpolation='nearest', aspect='auto', cmap=cm.RdBu, origin = 'lower')

#plt.plot(b)
#np.savetxt('E:\\MEASUREMENTS\\10mK_BlueFors_BFLD400\\RUN33\\run33\\csv_data\\2tone_0dBm.csv', b, delimiter=',')

In [292]:
plt.ion()

fig = plt.figure()
plt.imshow(pow_slice,interpolation='nearest', aspect='auto', cmap=cm.bone, origin = 'lower')
#np.save("E:\\MEASUREMENTS\\10mK_BlueFors_BFLD400\\RUN33\\run33\\CAmp\\spec_5dBm", pow_slice)
#ncs = Nsk_current_source("COM32")
#ncs.set_output_state(3,0)
#ncs.close()
#nwa.close()
#nwa.set_output_state("OFF")
#nwa.get_output_state()
#ncs.set_current(3,105e-6)

In [41]:
nwa = zvl13("TCPIP0::192.168.0.73::INSTR")
ncs = Nsk_current_source("COM5")
gen = N5173B_gen("TCPIP0::192.168.0.78::INSTR")

#  __AC-STARK SHIFT__

In [ ]:
nwa.set_output_state("ON")
nwa.set_standard_config({"power":-40,"nop":5,"averages": 1, "cf_span":[7.055083e9, 8e6], "bandwidth":10, "meas_counts":2})
nwa.set_timeout(500000)
gen = N5173B_gen("TCPIP0::192.168.0.78::INSTR")
gen.set_modulation_state(0)
gen.set_frequency(2.5e9)
gen.set_output_state("ON")
gen.get_output_state()
ncs.set_current(3,106e-6)#10uA step
gen.set_power(0)#power drive tone
#-10 + 7
gen_pow_list = 41
gen_freq_list = 601
plt.ion()

fig = plt.figure()

pow_slice = []
gen.set_power(0)
for j in range(gen_pow_list):
    nwa.set_power(-55 + j)
    time.sleep(1)
    temp_data = []
    onetone_data = []
    nwa_slice = []
    for i in range(gen_freq_list):
        gen.set_frequency(6.1e9+1e6*i)
        time.sleep(0.02)
        nwa.trig_meas()
        x_axis = nwa.get_X_axis_values()
        
        temp_data = nwa.get_data(int(nwa.get_meas_counts()))
        
        temp_dB = np.average(temp_data[0],0)
        
        #data = nwa.get_data(int(nwa.get_meas_counts()))
        
        x_upd = x_axis#[]
        y_upd = []
        nx = 101#x_upd.shape[0]
        no_labels = 5
        step_x = int(nx/(no_labels -1))
        x_positions = np.arange(0,nx,step_x)
        x_labels = x_upd[::step_x]
        plt.yticks(x_positions, x_labels)
        
        #z_upd.append(data[0][0])
        #y_upd.append(i)
        nwa_slice = temp_dB #[0][0]
        onetone_data.append(nwa_slice[2])
        
        #plt.imshow(np.transpose(z_upd),interpolation='nearest', aspect='auto', cmap=cm.RdBu)
        #nwa.set_timeout(1000)
        #plt.plot(x_axis, np.average(data[0],0))
        #line1.set_ydata(data[0][i][50])
        #fig.canvas.draw()
        #fig.canvas.flush_events()
    #print("Current freq - {}".format((j+1)*20e6+5e9)    
    pow_slice.append(onetone_data)
    plt.imshow(pow_slice,interpolation='nearest', aspect='auto', cmap=cm.bone, origin = 'lower')
    plt.title("Current freq - {}".format((j+1)*10e6+7e9))
    fig.canvas.draw()
    fig.canvas.flush_events()        
plt.colorbar()

nwa.set_output_state("OFF")
gen.set_output_state("OFF")

In [44]:
plt.imshow(pow_slice,interpolation='nearest', aspect='auto', cmap=cm.bone, origin = 'lower')
np.save("E:\\MEASUREMENTS\\10mK_BlueFors_BFLD400\\RUN33\\run33\\CAmp\\ac_Stark_overnight_prec_6GHz1-6GHz7_601p_-55pownwa-15_0powdr_.csv", pow_slice)

In [47]:
b = np.load("E:\\MEASUREMENTS\\10mK_BlueFors_BFLD400\\RUN33\\run33\\CAmp\\ac_Stark_overnight_prec_6GHz1-6GHz7_601p_-55pownwa-15_0powdr_.csv.npy")
plt.imshow(b, interpolation='nearest', aspect='auto', cmap=cm.RdBu, origin = 'lower')

#plt.plot(b)
np.savetxt('E:\\MEASUREMENTS\\10mK_BlueFors_BFLD400\\RUN33\\run33\\Camp\\ac_Stark_overnight_prec_6GHz1-6GHz7_601p_-55pownwa-15_0powdr_.csv.csv', b, delimiter=',')


In [98]:
print("Qubit frequency - {}Hz".format(int(5.5e9+20e6*52)))

Qubit frequency - 6540000000Hz


In [91]:
with open('12.04.19 13h36m_pulse.prn', 'wb') as f:
    pickle.dump(pow_slice, f)

In [106]:
with open('qubit3_twotone_-5dB.pickle', 'rb') as f:
    data_new = pickle.load(f)
fig = plt.figure()
plt.imshow(data_new, interpolation='nearest', aspect='auto', cmap=cm.bone, origin = 'lower')

In [10]:
act_data = data[0]
len(x_axis)

101

In [11]:
from __future__ import division
from matplotlib.patches import Patch
from pylab import *

y = np.linspace(0,1e-3,5)
x = x_axis
X,Y = meshgrid(y, x)

Z = np.transpose(np.array(act_data))

pcolor(X, Y, Z, cmap=cm.RdBu, vmax=abs(Z).max(), vmin=-abs(Z).max())
colorbar()
#axis([-3,3,-3,3])

plt.ion()
fig.canvas.draw()
fig.canvas.flush_events()

In [74]:
import matplotlib.animation as animation

fig2 = plt.figure()
x = x_axis
y = [np.arange(3)]
base = np.array(act_data[0:3])
X,Y = meshgrid(x, y)
ims = []
plt.pcolormesh(X, Y, base)

In [4]:
x = np.arange(100)
y = np.arange(100)
temp = [[0]*len(x)]*len(y)
nrows, ncols = 100, 100
grid = temp #.reshape((nrows, ncols))

for i in range(len(x)):
    temp[i] = np.random.rand(100)
fig = plt.figure()
plt.imshow(grid, extent=(x.min(), x.max(), y.max(), y.min()),
           interpolation='nearest', aspect='equal', cmap=cm.jet)
plt.colorbar()

plt.ion()
x_upd = []
y_upd = np.array([])
z_upd = []

for i in range(10):    
    time.sleep(1)
    #z_upd.append([1,2,3,4])
    z_upd.append(np.random.rand(4))
    plt.imshow(np.transpose(z_upd),interpolation='nearest', aspect='auto', cmap=cm.RdBu)
    
    fig.canvas.draw()
    fig.canvas.flush_events()    

In [2]:
def serial_ports():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result